### Disclaimer

Copyright and other regulations surrounding programmatic gathering of data vary by website and jurisdiction. It is your responsibility to check all applicable laws and terms of use of any website or service before attempting any web harvesting or similar activities.

# Application Programming Interface (API)

API usually refers to web based APIs

Data vendor provide consistent interface for data requests thus significantly reduce costs and efforts for clients to retrieve and maintain data fetching.


## Example - Singapore HDB Property Transactions

<img src="assets/HDB-Resale-screenshot.png" style="height: 400px"/>

Source: https://data.gov.sg/dataset/resale-flat-prices

Example Request: https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&limit=5

In [ ]:
import requests

In [ ]:
url = 'https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&limit=5'

resp = requests.get(url)
resp.text

Response from the http request is `text` in `json` format.

Parse the response data into python objects

In [ ]:
resp.json()

In [ ]:
import pandas as pd

pd.DataFrame(resp.json()['result']['records'])

# Example - US Census Data: Estimated Population

Register an API key: http://api.census.gov/data/key_signup.html

Geocoding: https://www.census.gov/geographies/reference-files/2019/demo/popest/2019-fips.html

API explained: https://www.census.gov/data/developers/guidance/api-user-guide/query-examples.html

Read example page: https://www.census.gov/data/developers/data-sets/popest-popproj/popest.html


In [ ]:
import requests

class CensusAPI_2019(object):
    def __init__(self, api_key):
        self.api_key = api_key
    
    def get_population(self, for_, in_=None, get_=['COUNTY','DATE_CODE','DATE_DESC','DENSITY','POP','NAME','STATE']):
        url = 'https://api.census.gov/data/2019/pep/population?'
        url += 'get={}'.format(','.join(get_))
        url += '&for={}'.format(for_)
        if in_:
            url += '&in={}'.format(in_)
        if self.api_key:
            url += '&key={}'.format(self.api_key)
        print(url)
        return requests.get(url).json()


In [ ]:
my_apikey = None

data = CensusAPI_2019(my_apikey).get_population(for_='county:*', in_='state:36')

data

In [ ]:
import pandas as pd

pd.DataFrame(data).T.set_index(0).T

## Example - UK Land Registration House Price Index

<img src="assets/UK-LandRegistry-screenshot.png" style="height: 500px"/>

#### Step 1: Locate the API urls

https://landregistry.data.gov.uk/   => `Price Index Data` => `details about the UKHPI data` => https://landregistry.data.gov.uk/app/doc/ukhpi

#### Step 2: Download monthly csv file

In [ ]:
import pandas as pd

url = 'https://landregistry.data.gov.uk/data/ukhpi/region/england-and-wales/month/2020-03.csv'

data = pd.read_csv(url)

In [ ]:
data

In [ ]:
data.T[0]['house price index']


#### Step 3: Build a function to automate the process

In [ ]:
import io
import datetime
import pandas as pd
import requests

def get_uk_monthly_hpi(region, year, month):
    url = 'https://landregistry.data.gov.uk/data/ukhpi/region/{region}/month/{year}-{month:02d}.csv'.format(
        region=region, year=year, month=month)
    
    data = pd.read_csv(url)
    
    return data.T[0]
    

In [ ]:
get_uk_monthly_hpi('london', 2021, 1)

In [ ]:
london_hpi = {}

for month in range(1,13):
    london_hpi[datetime.date(2020, month, 1)] = get_uk_monthly_hpi('london', 2020, month)

pd.DataFrame(london_hpi)

In [ ]:
ax = pd.DataFrame(london_hpi).T['house price index'].plot(figsize=(14, 8), title="London House Price Index")
ax.set_xlabel("Period")
ax.set_ylabel("House Price Index")